In [1]:
import os
import music21
import json

# us = environment.UserSettings()
# us.create()
# us['musescoreDirectPNGPath'] = 'Applications/'
# environment.set('musescoreDirectPNGPath', "./")
# music21.converter.program.musescoreDirectPNGPath =  "./"

In [2]:
# Setup

DATASET_PATH = "deutschl/test"
ACCEPTABLE_DURATIONS = [.25, .5, .75, 1, 1.5, 2, 3, 4]
SAVE_DIR = "./convertedFile"
SINGLE_FLE_DATASET = "./singleFileDataset"
MAPPING_PATH = "./mapping"
SEQUENCE_LENGTH = 64

In [3]:
def preprocess(dataset_pass):
    # load
    songs = loadSongs(dataset_pass)
    print(f"Number of loaded songs :{len(songs)}")

    for index, song in enumerate(songs): 
        # filter files by duration 
        if not hasAcceptableDuration(song, ACCEPTABLE_DURATIONS):
            continue

        # transpose to smae scale
        song = transpose(song)

        # time series representation encoding 
        encodedSong = encode(song)

        # save 
        save_path = os.path.join(SAVE_DIR, str(index))
        with open(save_path, "w") as fp: 
            fp.write(encodedSong)

def transpose(song):
    # key
    parts = song.getElementsByClass(music21.stream.Part)
    measure_part_zero = parts[0].getElementsByClass(music21.stream.Measure)
    key = measure_part_zero[0][4]

    if not isinstance(key, music21.key.Key):
        key = song.analyze("key")

    # interval transposition
    if key.mode == 'major' : 
        interval = music21.interval.Interval(key.tonic, music21.pitch.Pitch("C"))
    else:
        interval = music21.interval.Interval(key.tonic, music21.pitch.Pitch("A"))
    
    return song.transpose(interval)
    

def loadSongs(dataset_pass):
    songs = []
    for path, subdir, files in os.walk(dataset_pass):
        for file in files: 
            if file[-3:] == "krn":
                song = music21.converter.parse(os.path.join(path, file))
                songs.append(song)
    return songs

def hasAcceptableDuration(song, acceptables):
    for singleNote in song.flatten().notesAndRests:
        if singleNote.duration.quarterLength not in acceptables:
            return False

    return True

def encode(song, timestep=.25):
    encodedSong = []
    for event in song.flatten().notesAndRests:
        if isinstance(event, music21.note.Note):
            symbol = event.pitch.midi
            
        elif isinstance(event, music21.note.Rest):
            symbol = "r"

        # convert into timeseries notation
        steps = int(event.duration.quarterLength/timestep)
        for step in range(steps):
            if step == 0 :
                encodedSong.append(symbol)
            else: 
                encodedSong.append("_")
                
    encodedSong = " ".join(map(str, encodedSong))

    return encodedSong
    

In [7]:
def createSingleFileDataset(datasetPath, fileDatasetPath):
    newSongDelimiter = "/ " * SEQUENCE_LENGTH
    songs = ""

    for path, _, files in os.walk(datasetPath):
        for file in files:
            if file != ".DS_store":
                with open(os.path.join(path, file), "r") as fp: 
                    song = fp.read()
                songs = songs + song + " " + newSongDelimiter

    songs = songs[:-1]

    with open(fileDatasetPath, "w") as fp:
        fp.write(songs)

    return songs

def createMapping(songs):
    mapping = {}
    for index, symbol in enumerate(list(set(songs.split()))):
        mapping[symbol] = index

    with open(MAPPING_PATH, "w") as fp: 
        json.dump(mapping, fp)

In [ ]:
if __name__ == "__main__":
    preprocess(DATASET_PATH)
    songs = createSingleFileDataset(SAVE_DIR, SINGLE_FLE_DATASET)
    createMapping(songs)